In [ ]:
# https://towardsdatascience.com/introducing-transformers-interpret-explainable-ai-for-transformers-890a403a9470

In [2]:
# !pip install transformers-interpret

In [14]:
import torch

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [17]:
from transformers import AutoModel, AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import SequenceClassificationExplainer


model = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru")

In [10]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")

Downloading:   0%|          | 0.00/323 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [28]:
#Sentences we want sentence embeddings for
sentences = ['Привет! Как твои дела?',
             'Привет, Мурка, 42 твое любимое число?']


#Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=24, return_tensors='pt')

#Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

#Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

In [34]:
tokenizer.convert_tokens_to_ids('Привет,')

100

In [36]:
tokenizer.convert_ids_to_tokens(6571)

'привет'

In [32]:
tokenizer.get_vocab()

{'антидопинговое': 109035,
 'законодатель': 118003,
 '##няться': 7765,
 'соответствующей': 24865,
 'Обязательно': 33877,
 'убийством': 32153,
 'Газель': 74773,
 'Самаре': 35447,
 '##agram': 20418,
 'сдал': 28864,
 'обезвре': 72657,
 'Жесто': 53980,
 'появился': 6028,
 'перезагру': 105823,
 'космодром': 66322,
 'абы': 98254,
 'Давно': 20786,
 'депутатских': 66458,
 'спроса': 16753,
 'Ричардом': 67874,
 'регенерации': 99523,
 'нами': 4849,
 '##вшем': 7487,
 'нахожусь': 34584,
 'молний': 67081,
 'Сквозь': 32465,
 'Рикки': 76794,
 'лину': 86683,
 'комикса': 95708,
 'позднего': 38839,
 'профессором': 22330,
 'четверых': 32343,
 'танке': 62301,
 'крохотный': 77827,
 'Триполи': 52147,
 'прости': 7983,
 'принято': 5429,
 'Романов': 30702,
 'секретар': 20439,
 'загрязнение': 74193,
 'афри': 80344,
 'Томского': 77014,
 'Mich': 60666,
 'процедить': 67663,
 'козлом': 115863,
 'почётный': 64996,
 'неистреби': 116835,
 'Баки': 110911,
 'войдет': 29533,
 'сложение': 119684,
 'идентичным': 66981,
 'Ни

In [30]:
# indices of words
encoded_input

{'input_ids': tensor([[  101,  6571,   177,   785,  9229,  1196,   161,   102,     0,     0,
             0,     0],
        [  101,  6571,   121, 32429,   657,   121,  6397, 12498, 44900,  4420,
           161,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [29]:
sentence_embeddings

tensor([[ 0.4587, -0.2511, -0.4554,  ...,  0.3262, -0.9852,  0.2071],
        [ 0.7135, -0.1752, -0.0397,  ..., -0.2315, -0.7701,  0.4101]])

In [25]:
# SHAP for explaining text classification (but need to create the embedding in the model?)
# https://coderzcolumn.com/tutorials/artificial-intelligence/shap-values-for-text-classification-tasks

# https://shap.readthedocs.io/en/latest/example_notebooks/api_examples/plots/text.html
# https://shap.readthedocs.io/en/latest/example_notebooks/text_examples/sentiment_analysis/Positive%20vs.%20Negative%20Sentiment%20Classification.html